# Avaliação 2

### Questão 2
Utilizando a base de dados de preços de imóveis disponível no kaggle (https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data), apresente duas soluções de aprendizagem de máquina que consigam baixas taxas de erro médio.
* Lembre de fazer todo o pré-processamento, explicando suas decisões;
* Teste 3 variações de parâmetros dos métodos;
* Avalie os resultados usando uma métrica

#### Pré-processamento

É dado início ao pré-processamento, importando as bibliotecas necessárias e instanciando os databases forneceidos através do pandas.

É importante entender que:
* __sample__ equivale à instância do database que contém as saídas (coluna *SalePrice*) corretas para os testes
* __tr__ equivale à instância do database que contém os treinos
* __ts__ equivale à instância do database que contém os testes

Vale lembrar que o database __sample__ refere-se ao __teste__, e que o __teste__ não pertence ao __treino__. Portanto será interessante unir os dois databases futuramente.

In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing, cluster, neighbors, svm, metrics, tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import make_scorer, r2_score 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

sample = pd.read_csv("data/qt2/sample_submission.csv",sep=",")
tr = pd.read_csv("data/qt2/train.csv",sep=",")
ts = pd.read_csv("data/qt2/test.csv",sep=",")

#### Avaliando a quantidade de itens em cada database

Para entender a quantidade de itens com o qual se está trabalhando, é feita uma leitura através do comando __*len()*__.

In [2]:
print("Database de treino:", len(tr),"x",len(tr.columns))
print("Database de testes:", len(ts),"x",len(ts.columns))
print("Database que contém as saídas corretas do teste:", len(sample),"x",len(sample.columns))

Database de treino: 1460 x 81
Database de testes: 1459 x 80
Database que contém as saídas corretas do teste: 1459 x 2


#### Unindo todos os databases

Para passarem pelo pré-processamento, é interessante que todos os dados estejam juntos. Desta forma o __gs__ será unido ao __sample__, que será, por sua vez, unido ao __tr__.

❗️ __Utilizaremos o termo *dfTotal* para se referir ao DataFrame completo, que inclui o teste e o treino.__ ❗️

In [3]:
dfTotal = pd.concat([tr,pd.concat([sample, ts.drop(columns=['Id'])], axis=1)])

#### Tamanho resultante total

Para conferir a quantidade total de elementos no database resultante.

In [4]:
print(len(dfTotal),"x",len(dfTotal.columns))

2919 x 81


#### Analisando quantos itens nulos há no database

É importante conhecer quantos itens nulos existem no database para entender qual deve ser a medida adequada a se tomar mediante a quantidade de itens nulos.

In [5]:
print("-->Nulidade:")
for i in dfTotal.columns:
    print(i,"\t:",dfTotal[i].isnull().sum())

-->Nulidade:
Id 	: 0
MSSubClass 	: 0
MSZoning 	: 4
LotFrontage 	: 486
LotArea 	: 0
Street 	: 0
Alley 	: 2721
LotShape 	: 0
LandContour 	: 0
Utilities 	: 2
LotConfig 	: 0
LandSlope 	: 0
Neighborhood 	: 0
Condition1 	: 0
Condition2 	: 0
BldgType 	: 0
HouseStyle 	: 0
OverallQual 	: 0
OverallCond 	: 0
YearBuilt 	: 0
YearRemodAdd 	: 0
RoofStyle 	: 0
RoofMatl 	: 0
Exterior1st 	: 1
Exterior2nd 	: 1
MasVnrType 	: 24
MasVnrArea 	: 23
ExterQual 	: 0
ExterCond 	: 0
Foundation 	: 0
BsmtQual 	: 81
BsmtCond 	: 82
BsmtExposure 	: 82
BsmtFinType1 	: 79
BsmtFinSF1 	: 1
BsmtFinType2 	: 80
BsmtFinSF2 	: 1
BsmtUnfSF 	: 1
TotalBsmtSF 	: 1
Heating 	: 0
HeatingQC 	: 0
CentralAir 	: 0
Electrical 	: 1
1stFlrSF 	: 0
2ndFlrSF 	: 0
LowQualFinSF 	: 0
GrLivArea 	: 0
BsmtFullBath 	: 2
BsmtHalfBath 	: 2
FullBath 	: 0
HalfBath 	: 0
BedroomAbvGr 	: 0
KitchenAbvGr 	: 0
KitchenQual 	: 1
TotRmsAbvGrd 	: 0
Functional 	: 2
Fireplaces 	: 0
FireplaceQu 	: 1420
GarageType 	: 157
GarageYrBlt 	: 159
GarageFinish 	: 159
GarageCar

⚠️ __Houve muitas tabelas com valores nulos, vale nesta ocasião mostrar uma listagem para tornar o trabalho aqui desenvolvido o mais didático possível.__ ⚠️

In [6]:
amount = 0
for i in dfTotal.columns:
    if(dfTotal[i].isnull().sum()>0): 
        amount += 1
        print(i,"\t:",dfTotal[i].isnull().sum())
print("Quantidade total de tabelas que apresentaram valores nulos:",amount)

MSZoning 	: 4
LotFrontage 	: 486
Alley 	: 2721
Utilities 	: 2
Exterior1st 	: 1
Exterior2nd 	: 1
MasVnrType 	: 24
MasVnrArea 	: 23
BsmtQual 	: 81
BsmtCond 	: 82
BsmtExposure 	: 82
BsmtFinType1 	: 79
BsmtFinSF1 	: 1
BsmtFinType2 	: 80
BsmtFinSF2 	: 1
BsmtUnfSF 	: 1
TotalBsmtSF 	: 1
Electrical 	: 1
BsmtFullBath 	: 2
BsmtHalfBath 	: 2
KitchenQual 	: 1
Functional 	: 2
FireplaceQu 	: 1420
GarageType 	: 157
GarageYrBlt 	: 159
GarageFinish 	: 159
GarageCars 	: 1
GarageArea 	: 1
GarageQual 	: 159
GarageCond 	: 159
PoolQC 	: 2909
Fence 	: 2348
MiscFeature 	: 2814
SaleType 	: 1
Quantidade total de tabelas que apresentaram valores nulos: 34


⚠️ __34 colunas apresentaram valores nulos. A essa altura é importante tratar de cada uma individualmente.__ ⚠️

#### Tratando as nulidades

Algumas colunas apresentaram uma quantidade muito alta de valores nulos. Entendeu-se, desta forma, como mais adequada a decisão de apagar essas colunas, uma vez que média, desvio padrão ou moda são valores que perdem significado dada uma quantidade tão alta de valores nulos. Sendo assim, __as colunas__:

* __MiscFeature__ (com 96% dos valores nulos)
* __PoolQC__ (com 99% dos valores nulos)
* __Fence__ (com 80% dos valores nulos)
* __FireplaceQu__ (com 48% dos valores nulos)
* __Alley__ (com 93% dos valores nulos)

__serão excluídas__.

As colunas que apresentam uma quantidade desprezível de dados faltantes:

* __MSZoning__
* __Utilities__
* __Exterior1st__
* __Exterior2nd__
* __BsmtFinSF1__
* __BsmtFinSF2__
* __BsmtUnfSF__
* __TotalBsmtSF__
* __Electrical__
* __BsmtFullBath__
* __BsmtHalfBath__
* __KitchenQual__
* __Functional__
* __GarageCars__
* __GarageArea__ 
* __SaleType__

__terão as linhas que apresenta esses dados faltantes excluídas__.

As demais colunas __serão preenchidas com a mediana (no caso de colunas numéricas) ou moda (no caso de colunas qualitativas) dos valores faltantes na devida coluna__:

* __LotFrontage__
* __MasVnrType__
* __MasVnrArea__
* __BsmtQual__
* __BsmtCond__
* __BsmtExposure__
* __BsmtFinType1__
* __BsmtFinType2__
* __GarageType__
* __GarageYrBlt__
* __GarageFinish__
* __GarageQual__
* __GarageCond__

##### Tratando colunas com alta quantidade de valores nulos

In [7]:
dfTotal = dfTotal.drop(columns=['MiscFeature','PoolQC','Fence','FireplaceQu','Alley'])
print(len(dfTotal),"x",len(dfTotal.columns))

2919 x 76


##### Tratando colunas com quantidade ínfima de valores nulos

In [8]:
dfTotal = dfTotal.dropna(subset=['MSZoning','Utilities','Exterior1st','Exterior2nd','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','Electrical','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional','GarageCars','GarageArea','SaleType'])
print(len(dfTotal),"x",len(dfTotal.columns))

2906 x 76


##### Tratando demais colunas

In [9]:
dfTotal.fillna(dfTotal['LotFrontage'].median(),inplace=True)
print("LotFrontage\t:",dfTotal['LotFrontage'].isnull().sum())

LotFrontage	: 0


In [10]:
dfTotal.fillna(dfTotal['MasVnrType'].mode(),inplace=True)
print("MasVnrType\t:",dfTotal['MasVnrType'].isnull().sum())

MasVnrType	: 0


In [11]:
dfTotal.fillna(dfTotal['MasVnrArea'].median(),inplace=True)
print("MasVnrArea\t:",dfTotal['MasVnrArea'].isnull().sum())

MasVnrArea	: 0


In [12]:
dfTotal.fillna(dfTotal['BsmtQual'].mode(),inplace=True)
print("BsmtQual\t:",dfTotal['BsmtQual'].isnull().sum())

BsmtQual	: 0


In [13]:
dfTotal.fillna(dfTotal['BsmtCond'].mode(),inplace=True)
print("BsmtCond\t:",dfTotal['BsmtCond'].isnull().sum())

BsmtCond	: 0


In [14]:
dfTotal.fillna(dfTotal['BsmtExposure'].mode(),inplace=True)
print("BsmtExposure\t:",dfTotal['BsmtExposure'].isnull().sum())

BsmtExposure	: 0


In [16]:
dfTotal.fillna(dfTotal['BsmtFinType1'].mode(),inplace=True)
print("BsmtFinType1\t:",dfTotal['BsmtFinType1'].isnull().sum())

BsmtFinType1	: 0


In [20]:
dfTotal.fillna(dfTotal['BsmtFinType2'].mode(),inplace=True)
print("BsmtFinType2\t:",dfTotal['BsmtFinType2'].isnull().sum())

BsmtFinType2	: 0


In [22]:
dfTotal.fillna(dfTotal['GarageType'].mode(),inplace=True)
print("GarageType\t:",dfTotal['GarageType'].isnull().sum())

GarageType	: 0


In [27]:
dfTotal.fillna(dfTotal['GarageYrBlt'].median(),inplace=True)
print("GarageYrBlt\t:",dfTotal['GarageYrBlt'].isnull().sum())

GarageYrBlt	: 0


In [37]:
dfTotal.fillna(dfTotal['GarageFinish'].mode(),inplace=True)
print("GarageFinish\t:",dfTotal['GarageFinish'].isnull().sum())

GarageFinish	: 0


In [35]:
dfTotal.fillna(dfTotal['GarageQual'].mode(),inplace=True)
print("GarageQual\t:",dfTotal['GarageQual'].isnull().sum())

GarageQual	: 0


In [39]:
dfTotal.fillna(dfTotal['GarageCond'].mode(),inplace=True)
print("GarageCond\t:",dfTotal['GarageCond'].isnull().sum())

GarageCond	: 0


In [42]:
print(len(dfTotal),"x",len(dfTotal.columns))

2906 x 76


#### Conferindo quantidade de itens nulos no database

In [41]:
dfTotal.isnull().sum().sum()

0

✅ __Foi feita com sucesso a limpeza de valores nulos nas tabelas.__ ✅

❗️ __Entretanto é necessário um cuidado. Há muitas colunas, entretanto, com valores qualitativos. Muito embora essas colunas qualitativas não apresentem valores nulos após a limpeza, podem conter células que apresentem conteúdo que representam o "vazio", que deve ser realizada a interpretação se esse "vazio" é proposital com a lógica do database ou se é tem o mesmo significado lógico que um nulo. Para isso, como será feito adiante, será analizado adiante quais tabelas são qualitativas e se possuem valores que representam o "vazio" supracitado.__ ❗️

In [46]:
print("Colunas qualitativas: ")
amount = 0
for i in dfTotal.columns:
    if(dfTotal.dtypes[i]=='object'):
        amount += 1
        print(amount,"-\t",i)

Colunas qualitativas: 
1 -	 MSZoning
2 -	 Street
3 -	 LotShape
4 -	 LandContour
5 -	 Utilities
6 -	 LotConfig
7 -	 LandSlope
8 -	 Neighborhood
9 -	 Condition1
10 -	 Condition2
11 -	 BldgType
12 -	 HouseStyle
13 -	 RoofStyle
14 -	 RoofMatl
15 -	 Exterior1st
16 -	 Exterior2nd
17 -	 MasVnrType
18 -	 ExterQual
19 -	 ExterCond
20 -	 Foundation
21 -	 BsmtQual
22 -	 BsmtCond
23 -	 BsmtExposure
24 -	 BsmtFinType1
25 -	 BsmtFinType2
26 -	 Heating
27 -	 HeatingQC
28 -	 CentralAir
29 -	 Electrical
30 -	 KitchenQual
31 -	 Functional
32 -	 GarageType
33 -	 GarageFinish
34 -	 GarageQual
35 -	 GarageCond
36 -	 PavedDrive
37 -	 SaleType
38 -	 SaleCondition
